# Flyweight Pattern

## Intent
Use sharing to support large numbers of fine-grained objects efficiently by minimizing memory usage.

## Problem
Your application needs many objects but:
- Creating them all consumes too much memory
- Many objects share common data
- Most object state can be made extrinsic
- Application doesn't depend on object identity

**Real-world analogy**: Chess pieces - white king and black king share the same 3D model (intrinsic), but have different positions (extrinsic)

## When to Use
✅ **Use when:**
- Application uses large number of objects
- Storage costs are high due to quantity
- Most object state can be extrinsic
- Objects can be grouped by intrinsic state
- Application doesn't depend on object identity

❌ **Avoid when:**
- Few objects are needed
- No shared state exists
- Extrinsic state is hard to compute/maintain

## Pattern Structure
```
┌────────────────┐     ┌──────────┐
│FlyweightFactory│────►│Flyweight │
├────────────────┤     ├──────────┤
│getFlyweight()  │     │operation()│
│flyweights[]    │     └──────────┘
└────────────────┘           ▲
                             │
                    ┌────────┴────────┐
              ┌─────┴─────┐    ┌──────┴──────┐
              │Shared     │    │Unshared     │
              │Flyweight  │    │Flyweight    │
              └───────────┘    └─────────────┘
```

## Key Concepts

**Intrinsic State**: Shared, stored in flyweight (e.g., font, texture)
**Extrinsic State**: Unique, passed to methods (e.g., position, color)

## Example 1: Without Flyweight

**Problem**: High memory usage from duplicated data

In [ ]:
import sys

# WITHOUT Flyweight - Each tree stores all data
class Tree:
    def __init__(self, x, y, tree_type, texture, mesh):
        self.x = x
        self.y = y
        self.tree_type = tree_type  # "Oak", "Pine", etc.
        self.texture = texture  # Large texture data
        self.mesh = mesh  # 3D model data
    
    def draw(self):
        print(f"Drawing {self.tree_type} at ({self.x}, {self.y})")

# Create forest
forest = []
oak_texture = "[OAK TEXTURE DATA " + "X" * 100 + "]"
oak_mesh = "[OAK 3D MODEL " + "Y" * 100 + "]"

for i in range(1000):
    # Each tree duplicates texture and mesh!
    tree = Tree(i % 100, i // 100, "Oak", oak_texture, oak_mesh)
    forest.append(tree)

memory = sum(sys.getsizeof(vars(tree)) for tree in forest[:10])
print(f"\n❌ Memory for 10 trees: ~{memory:,} bytes")
print(f"❌ Each tree stores full texture and mesh!")
print(f"❌ 1000 trees = MASSIVE memory waste!")

## Implementation: Flyweight Pattern

In [ ]:
from typing import Dict

# Flyweight: Shared state (intrinsic)
class TreeType:
    """Flyweight storing shared tree data."""
    
    def __init__(self, name: str, color: str, texture: str, mesh: str):
        self.name = name  # Intrinsic
        self.color = color  # Intrinsic
        self.texture = texture  # Intrinsic (large data!)
        self.mesh = mesh  # Intrinsic (large data!)
    
    def draw(self, x: int, y: int) -> None:
        """Draw tree at position (extrinsic state)."""
        print(f"  🌳 {self.name} ({self.color}) at ({x}, {y})")


# Flyweight Factory
class TreeFactory:
    """Creates and caches flyweights."""
    
    _tree_types: Dict[str, TreeType] = {}
    
    @classmethod
    def get_tree_type(cls, name: str, color: str, texture: str, mesh: str) -> TreeType:
        """Get or create flyweight."""
        key = f"{name}_{color}"
        
        if key not in cls._tree_types:
            print(f"  ➕ Creating new TreeType: {key}")
            cls._tree_types[key] = TreeType(name, color, texture, mesh)
        else:
            print(f"  ♻️  Reusing existing TreeType: {key}")
        
        return cls._tree_types[key]
    
    @classmethod
    def get_flyweight_count(cls) -> int:
        return len(cls._tree_types)


# Context: Unique state (extrinsic)
class Tree:
    """Individual tree with position (extrinsic state)."""
    
    def __init__(self, x: int, y: int, tree_type: TreeType):
        self.x = x  # Extrinsic
        self.y = y  # Extrinsic
        self.tree_type = tree_type  # Reference to flyweight
    
    def draw(self) -> None:
        self.tree_type.draw(self.x, self.y)


# Forest: Client code
class Forest:
    """Manages trees using flyweights."""
    
    def __init__(self):
        self.trees = []
    
    def plant_tree(self, x: int, y: int, name: str, color: str, texture: str, mesh: str) -> None:
        tree_type = TreeFactory.get_tree_type(name, color, texture, mesh)
        tree = Tree(x, y, tree_type)
        self.trees.append(tree)
    
    def draw(self) -> None:
        for tree in self.trees:
            tree.draw()


# Demo
print("\n=== Flyweight Pattern ===")

forest = Forest()

print("\n1. Planting trees:")
oak_texture = "[OAK TEXTURE]" + "X" * 100
oak_mesh = "[OAK MODEL]" + "Y" * 100
pine_texture = "[PINE TEXTURE]" + "X" * 100
pine_mesh = "[PINE MODEL]" + "Y" * 100

# Plant 5 oaks
for i in range(5):
    forest.plant_tree(i * 10, 20, "Oak", "green", oak_texture, oak_mesh)

# Plant 3 pines
for i in range(3):
    forest.plant_tree(i * 10, 40, "Pine", "dark green", pine_texture, pine_mesh)

print(f"\n2. Total trees: {len(forest.trees)}")
print(f"   Flyweights created: {TreeFactory.get_flyweight_count()}")

print("\n3. Drawing forest:")
forest.draw()

print(f"\n✅ 8 trees share only 2 flyweights!")
print(f"✅ Memory saved by reusing textures and meshes!")

## Real-World Example: Text Editor Characters

In [ ]:
# Flyweight: Character formatting (intrinsic)
class CharacterStyle:
    """Flyweight for character formatting."""
    
    def __init__(self, font: str, size: int, color: str):
        self.font = font
        self.size = size
        self.color = color
        # Font would contain large font data
    
    def render(self, char: str, x: int, y: int) -> None:
        print(f"  [{self.font} {self.size}pt {self.color}] '{char}' at ({x},{y})")


# Flyweight Factory
class StyleFactory:
    _styles: Dict[str, CharacterStyle] = {}
    
    @classmethod
    def get_style(cls, font: str, size: int, color: str) -> CharacterStyle:
        key = f"{font}_{size}_{color}"
        
        if key not in cls._styles:
            cls._styles[key] = CharacterStyle(font, size, color)
        
        return cls._styles[key]
    
    @classmethod
    def get_style_count(cls) -> int:
        return len(cls._styles)


# Context: Individual character (extrinsic state)
class Character:
    def __init__(self, char: str, x: int, y: int, style: CharacterStyle):
        self.char = char  # Extrinsic
        self.x = x  # Extrinsic
        self.y = y  # Extrinsic
        self.style = style  # Flyweight
    
    def render(self) -> None:
        self.style.render(self.char, self.x, self.y)


# Document
class TextEditor:
    def __init__(self):
        self.characters = []
    
    def insert_char(self, char: str, x: int, y: int, font: str, size: int, color: str) -> None:
        style = StyleFactory.get_style(font, size, color)
        character = Character(char, x, y, style)
        self.characters.append(character)
    
    def render(self) -> None:
        for char in self.characters:
            char.render()


# Demo
print("\n=== Text Editor Flyweight ===")

editor = TextEditor()

# Type "Hello World" with formatting
print("\n1. Typing 'Hello World':")
text = "Hello World"
for i, char in enumerate(text):
    if char == 'H' or char == 'W':
        editor.insert_char(char, i * 10, 0, "Arial", 14, "red")
    elif char == ' ':
        continue
    else:
        editor.insert_char(char, i * 10, 0, "Arial", 12, "black")

print(f"\n2. Characters: {len(editor.characters)}")
print(f"   Styles created: {StyleFactory.get_style_count()}")

print("\n3. Rendering document:")
editor.render()

print(f"\n✅ 10 characters share only {StyleFactory.get_style_count()} styles!")

## Real-World Example: Game Particle System

In [ ]:
import random

# Flyweight: Particle type (intrinsic)
class ParticleType:
    """Shared particle properties."""
    
    def __init__(self, name: str, color: str, texture: str):
        self.name = name
        self.color = color
        self.texture = texture  # Large texture data
        # Physics properties, etc.
    
    def render(self, x: float, y: float, velocity_x: float, velocity_y: float) -> None:
        print(f"  {self.color} {self.name} at ({x:.1f},{y:.1f}) moving ({velocity_x:.1f},{velocity_y:.1f})")


# Factory
class ParticleFactory:
    _types: Dict[str, ParticleType] = {}
    
    @classmethod
    def get_particle_type(cls, name: str, color: str, texture: str) -> ParticleType:
        key = name
        if key not in cls._types:
            cls._types[key] = ParticleType(name, color, texture)
        return cls._types[key]


# Context: Individual particle (extrinsic)
class Particle:
    def __init__(self, x: float, y: float, velocity_x: float, velocity_y: float, particle_type: ParticleType):
        self.x = x
        self.y = y
        self.velocity_x = velocity_x
        self.velocity_y = velocity_y
        self.particle_type = particle_type
    
    def update(self, dt: float) -> None:
        self.x += self.velocity_x * dt
        self.y += self.velocity_y * dt
    
    def render(self) -> None:
        self.particle_type.render(self.x, self.y, self.velocity_x, self.velocity_y)


# Particle system
class ParticleSystem:
    def __init__(self):
        self.particles = []
    
    def emit(self, x: float, y: float, particle_type_name: str) -> None:
        # Get or create particle type
        if particle_type_name == "fire":
            particle_type = ParticleFactory.get_particle_type("fire", "🔥 red", "[FIRE_TEX]")
        elif particle_type_name == "smoke":
            particle_type = ParticleFactory.get_particle_type("smoke", "💨 gray", "[SMOKE_TEX]")
        elif particle_type_name == "spark":
            particle_type = ParticleFactory.get_particle_type("spark", "✨ yellow", "[SPARK_TEX]")
        else:
            return
        
        # Create particle with random velocity
        particle = Particle(
            x, y,
            random.uniform(-5, 5),
            random.uniform(-5, 5),
            particle_type
        )
        self.particles.append(particle)
    
    def update(self, dt: float) -> None:
        for particle in self.particles:
            particle.update(dt)
    
    def render(self) -> None:
        for particle in self.particles[:5]:  # Show first 5
            particle.render()


# Demo
print("\n=== Particle System Flyweight ===")

system = ParticleSystem()

print("\n1. Emitting particles:")
# Emit explosion
for i in range(50):
    system.emit(100, 100, "fire")
for i in range(30):
    system.emit(100, 100, "smoke")
for i in range(20):
    system.emit(100, 100, "spark")

print(f"   Total particles: {len(system.particles)}")
print(f"   Particle types: {len(ParticleFactory._types)}")

print("\n2. First frame (showing first 5):")
system.render()

print("\n3. After update (dt=0.1):")
system.update(0.1)
system.render()

print(f"\n✅ 100 particles share only 3 particle types!")
print(f"✅ Massive memory savings for particle effects!")

## Advantages & Disadvantages

### ✅ Advantages
1. **Memory efficiency**: Share common data across many objects
2. **Performance**: Reduce memory allocation overhead
3. **Scalability**: Handle large numbers of objects
4. **Immutability**: Flyweights are typically immutable

### ❌ Disadvantages
1. **Complexity**: More complex code
2. **Extrinsic state**: Must compute/pass extrinsic state
3. **CPU vs Memory**: Trade CPU time for memory
4. **Thread safety**: Shared objects need synchronization

## Memory Comparison

**Without Flyweight:**
```
1000 trees × (position + type + texture + mesh)
= 1000 × (8 + 8 + 1KB + 10KB)
= ~11 MB
```

**With Flyweight:**
```
1000 trees × position + 2 types × (type + texture + mesh)
= 1000 × 8 + 2 × (8 + 1KB + 10KB)
= ~30 KB
```

## Common Use Cases

1. **Text editors**: Character formatting
2. **Games**: Trees, grass, particles, bullets
3. **UI**: Icons, cursors
4. **Graphics**: Shapes with shared properties
5. **String interning**: Reuse string objects
6. **Connection pooling**: Reuse database connections
7. **Caching**: Cache frequently used objects

## Related Patterns

- **Singleton**: Factory can be singleton
- **Factory**: Creates flyweights
- **State/Strategy**: Can be implemented as flyweights
- **Composite**: Often combined with Flyweight

## Best Practices

1. **Identify shared state**: Separate intrinsic from extrinsic
2. **Make flyweights immutable**: Prevent accidental changes
3. **Use factory**: Centralize flyweight creation
4. **Cache flyweights**: Reuse instead of recreate
5. **Consider thread safety**: Synchronize if needed
6. **Profile first**: Only optimize if memory is actually a problem
7. **Document clearly**: Explain intrinsic vs extrinsic state

## Python-Specific: String Interning

Python uses flyweight pattern for string interning:

```python
a = "hello"
b = "hello"
a is b  # True! Same object in memory
```

## Summary

Flyweight pattern enables:
- Efficient memory usage for many similar objects
- Sharing intrinsic state
- Passing extrinsic state as parameters
- Scalability to millions of objects

Perfect for: Games (particles, terrain), text editors, graphics, large object collections.

**Key Insight**: Share common data (intrinsic state) among many objects, while keeping unique data (extrinsic state) separate, dramatically reducing memory usage!